In [50]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd

1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.)

In [2]:
# Класс узла
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index 
        self.t = t
        self.true_branch = true_branch
        self.false_branch = false_branch

In [3]:
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
    
    def predict(self):
        # Подсчет количества объектов разных классов
        classes = {}
        
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        
        # Найдем класс, с максимальным количеством объектов в листе и вернем его
        prediction = max(classes, key=classes.get)
        return prediction

In [4]:
# Расчет критерия Джини
def gini(labels):
    classes = {}
    for label in labels:
        
        if label not in classes:
            classes[label] = 0
        
        classes[label] += 1
        
    # расчет критерия
    
    impurity = 1
    
    for label in classes:
        
        p = classes[label] / len(labels)
        impurity -= p ** 2
    
    return impurity

In [5]:
# Расчет прироста

def gain(left_labels, right_labels, root_gini):
    
    # Доля воборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [6]:
# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
    
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
    
    return true_data, false_data, true_labels, false_labels

In [116]:
# Нахождение наилучшего разбиения

# ВСЕ ОСНОВНЫЕ ИЗМЕНЕНИЯ БЫЛИ В ЭТОЙ ФУНКЦИИ

def find_best_split(data, labels, n_features, min_samples_leaf=None):
    
    # Обозначим минимальное количество объектов в узле
    
    # Изменил
#     min_samples_leaf = 5
    
    root_gini = gini(labels)
    
    best_gain = 0
    best_t = None
    best_index = None
    
    # Изменил
#     n_features = data.shape[1]
    
    for index in range(n_features):
        
        # Будем проверять только уникальные значения признака, исключая повторения
        
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            
            # Пропускаем разбиения, в которых в узле остается менее 5 объектов
            
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_labels, false_labels, root_gini)
            
            # выбираем порог, на котором получается максимальный прирост качества
            
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index
    
    return best_gain, best_t, best_index

In [117]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, n_features, min_n_leaf):
    
    gain, t, index = find_best_split(data, labels, n_features, min_n_leaf)
    
    # Базовый случай - прекращаем рекурсию, когда нет прироста в качестве
    if gain == 0:
        return Leaf(data, labels)
    
    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
    
    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, n_features, min_n_leaf)
    false_branch = build_tree(false_data, false_labels, n_features, min_n_leaf)
    
    return Node(index, t, true_branch, false_branch)

In [118]:
def classify_object(obj, node):
    
    # Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer
    
    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [119]:
def predict(data, tree):
    classes = []
    
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [120]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100

In [121]:
def best_choice(data, labels, n_features, min_leafs):
    all_final_data = []
    for i in n_features:
        for j in min_leafs:
            tree = build_tree(data, labels, i, j)
            answer = predict(data, tree)
            acc = accuracy_metric(labels, answer)
            all_final_data.append((i, j, acc))
    
    return all_final_data        

In [122]:
classification_data, classification_labels = datasets.make_classification(n_features=2, 
                                                                          n_informative=2, 
                                                                          n_classes=2, 
                                                                          n_redundant=0, 
                                                                          n_clusters_per_class=1, 
                                                                          random_state=5)

In [123]:
train_data, test_data, train_labels, test_labels = train_test_split(classification_data, 
                                                                    classification_labels, 
                                                                    test_size=0.3, 
                                                                    random_state=1)

In [124]:
n_features = [i for i in range(train_data.shape[1] + 1)]
min_leafs = [i for i in range(10)]
all_trees = best_choice(train_data, train_labels, n_features, min_leafs)
df = pd.DataFrame(all_trees, columns=['n_features', 'min_leafs', 'accuracy_metric'])
df

<ipython-input-108-35d8370b7f5b>:2: RuntimeWarning: Mean of empty slice.
  mean = targets.mean()
<ipython-input-108-35d8370b7f5b>:3: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((targets - mean)**2)/targets.shape[0]


,n_features,min_leafs,accuracy_metric
0,0,0,0.000000
1,0,1,0.000000
2,0,2,0.000000
3,0,3,0.000000
4,0,4,0.000000
5,0,5,0.000000
6,0,6,0.000000
7,0,7,0.000000
8,0,8,0.000000
9,0,9,0.000000


2. *Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [133]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index
        self.t = t
        self.true_branch = true_branch
        self.false_branch = false_branch

In [134]:
class Leaf:
    
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        self.prediction = self.predict()
        
    def predict(self):
        return self.targets.mean()

In [135]:
def find_best_split(data, labels, n_features, min_samples_leaf=None):
    
    # Обозначим минимальное количество объектов в узле
    
    # Изменил
#     min_samples_leaf = 5
    
    root_mse = mse(labels)
    
    best_gain = 0
    best_t = None
    best_index = None
    
    # Изменил
#     n_features = data.shape[1]
    
    for index in range(n_features):
        
        # Будем проверять только уникальные значения признака, исключая повторения
        
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            
            # Пропускаем разбиения, в которых в узле остается менее 5 объектов
            
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            current_gain = gain(true_labels, false_labels, root_mse)
            
            # выбираем порог, на котором получается максимальный прирост качества
            
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index
    
    return best_gain, best_t, best_index

In [136]:
def mse(targets):
    mean = targets.mean()
    return np.sum((targets - mean)**2)/targets.shape[0]

In [137]:
def gain(left_targets, right_targets, root_mse):
    
    p = float(len(left_targets)/(len(left_targets) + len(right_targets)))
    
    return root_mse - p * mse(left_targets) - (1 - p) * mse(right_targets)

In [138]:
def accuracy_metric(actual, predicted):
    return np.mean((actual - predicted)**2)

In [139]:
X, y = datasets.make_regression(n_samples=100, 
                                n_features=2, 
                                n_informative=2, 
                                n_targets=1, 
                                noise=5, 
                                random_state=1)

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [141]:
n_features = [i for i in range(X_train.shape[1] + 1)]
min_leafs = [i for i in range(10)]
reg_all_trees = best_choice(X_train, y_train, n_features, min_leafs)
reg_all_trees

<ipython-input-136-35d8370b7f5b>:2: RuntimeWarning: Mean of empty slice.
  mean = targets.mean()
<ipython-input-136-35d8370b7f5b>:3: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum((targets - mean)**2)/targets.shape[0]


[(0, 0, 5831.845389707),
 (0, 1, 5831.845389707),
 (0, 2, 5831.845389707),
 (0, 3, 5831.845389707),
 (0, 4, 5831.845389707),
 (0, 5, 5831.845389707),
 (0, 6, 5831.845389707),
 (0, 7, 5831.845389707),
 (0, 8, 5831.845389707),
 (0, 9, 5831.845389707),
 (1, 0, 0.0),
 (1, 1, 0.0),
 (1, 2, 1957.4936418744269),
 (1, 3, 2531.8682815680586),
 (1, 4, 3916.2693749320883),
 (1, 5, 4555.2221830448725),
 (1, 6, 4717.609424717837),
 (1, 7, 4808.158701550357),
 (1, 8, 5045.066111194828),
 (1, 9, 5045.066111194828),
 (2, 0, 0.0),
 (2, 1, 0.0),
 (2, 2, 54.8313914189093),
 (2, 3, 138.45939531910986),
 (2, 4, 174.27024227197023),
 (2, 5, 461.5015604885304),
 (2, 6, 524.6114479076426),
 (2, 7, 553.5834944091387),
 (2, 8, 607.1949044116833),
 (2, 9, 652.3710371786353)]